<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: simple_ppm.c

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the functions `interpolate_for_A_gauge_rhs` and `calculate_phitilde_and_A_gauge_rhs`. These functions provide the ability to calculate the right-hand sides for $\tilde{\Phi}$ and the gauge contribution to $A_i$.

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#interpolate_for_A_gauge_rhs): **The `interpolate_for_A_gauge_rhs` function**
1. [Step 3](#calculate_phitilde_and_A_gauge_rhs): **The `calculate_phitilde_and_A_gauge_rhs` function**
1. [Step 4](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss the functions which facillitate the calculation of the right-hand sides for $\tilde{\Phi}$ and the gauge contribution to $A_i$. The `interpolate_for_A_gauge_rhs` function takes stencils of various quantities and computes intermediate quantities which are used in the `calculate_phitilde_and_A_gauge_rhs` function. These are separated like this to allow for OMP parallelism, as the second function needs stencils of the intermediate quantities computed by `interpolate_for_A_gauge_rhs`.

<a id='interpolate_for_A_gauge_rhs'></a>

# Step 2: The `interpolate_for_A_gauge_rhs` function \[Back to [top](#toc)\]

$$\label{interpolate_for_A_gauge_rhs}$$


First, let's look at the arguments of `interpolate_for_A_gauge_rhs`:
```
void interpolate_for_A_gauge_rhs(
      A_gauge_vars *restrict gauge_vars,
      A_gauge_rhs_vars *restrict gauge_rhs_vars)
```

The first argument is a struct containing stencils for the needed input variables. The second argument is a struct containing the output quantities. The input struct is defined by
```
typedef struct A_gauge_vars {
  double gupxx[2][2][2];
  double gupxy[2][2][2];
  double gupxz[2][2][2];
  double gupyy[2][2][2];
  double gupyz[2][2][2];
  double gupzz[2][2][2];
  double lapse[2][2][2];
  double psi[2][2][2];
  double shiftx[2][2][2];
  double shifty[2][2][2];
  double shiftz[2][2][2];
  double A_x[3][3][3];
  double A_y[3][3][3];
  double A_z[3][3][3];
  double phitilde;
} A_gauge_vars;
```
The `gup*` arrays contain stencils of the raised BSSN metric $g^{ij}$. `lapse`, `psi`, and `shift*` are stencils for the other metric quantities. These all have the same stenciling: for the point $(i,j,k)$, the stencil goes from $(i,j,k)$ to $(i+2,j+2,k+2)$.

The `A_*` arrays contain stencils for $A_i$ and have stencils from $i-1$ to $i+1$ for each index. Technically, the function only uses the stencils
```
A_x: { 0,1,-1,1,-1,1}
A_y: {-1,1, 0,1,-1,1}
A_z: {-1,1,-1,1, 0,1}
```
but making the arrays cubes allows for easier filling of the stencil.

Finally, `phitilde` just contains the value of $\tilde{\Phi}(i,j,k)$.

The output values return inside the struct
```
typedef struct A_gauge_rhs_vars {
  double dxi[3];
  double alpha_interp;
  double alpha_Phi_minus_betaj_A_j_interp[4];
  double alpha_sqrtg_Ax_interp[2];
  double alpha_sqrtg_Ay_interp[2];
  double alpha_sqrtg_Az_interp[2];
  double phitildex[5], shiftx_interp[5];
  double phitildey[5], shifty_interp[5];
  double phitildez[5], shiftz_interp[5];
  double phitilde_rhs, A_x_gauge_rhs, A_y_gauge_rhs, A_z_gauge_rhs;
} A_gauge_rhs_vars;
```
While this struct does have stencils, those are not used in the `interpolate_for_A_gauge_rhs` function. We reuse this struct for the `calculate_phitilde_and_A_gauge_rhs` function. The returned interpolated quantities are `alpha_interp`, `alpha_sqrtg_A*_interp`, `alpha_Phi_minus_betaj_A_j_interp`, and `shift*_interp`. For the arrays, the returned value is stored in the first (index 0) element of the arrays.

The actual function interpolates these quantities to the points needed for computing the right-hand sides. The interpolations are done through a simple averaging function. First, $\alpha$ is interpolated to the point $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$, which is stored in `alpha_interp`. Then, the quantity
$$
\alpha \sqrt{g} A^x
$$
is computed at the point $(i,j+\frac{1}{2},k+\frac{1}{2})$. The metric, $\alpha \psi^2$, $A^y$, and $A^z$ are interpolated to this point, while A^x naturally lives at this point. Using the interpolated quantities,
$$
\alpha \sqrt{g} A^x = \alpha \psi^2 (g^{xx}A_x + g^{xy}A_y + g^{xz}A_z)
$$
The other two quantities are similar, with the $A^y$ term living at the point $(i+\frac{1}{2},j,k+\frac{1}{2})$ and the $A^z$ term living at the point $(i+\frac{1}{2},j+\frac{1}{2},k)$. The expressions for them are

\begin{align}
\alpha \sqrt{g} A^y &= \alpha \psi^2 (g^{xy}A_x + g^{yy}A_y + g^{yz}A_z) \\
\alpha \sqrt{g} A^z &= \alpha \psi^2 (g^{xz}A_x + g^{yz}A_y + g^{zz}A_z)
\end{align}

Next, we set the `shift*_interp` variables by just interpolating the shift to the point $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$. Finally, we compute
$$
\alpha \Phi - \beta^j A_j = \frac{\alpha}{\psi^6}\tilde{\Phi} - \beta^j A_j
$$
also at the point $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$.

<a id='calculate_phitilde_and_A_gauge_rhs'></a>

# Step 3: The `calculate_phitilde_and_A_gauge_rhs` Function \[Back to [top](#toc)\]

$$\label{calculate_phitilde_and_A_gauge_rhs}$$

The function which actually computes the right-hand sides is the `calculate_phitilde_and_A_gauge_rhs` function. The `interpolate_for_A_gauge_rhs` function just provides a convenient function to compute the inputs for this function from more standard variables. The function
```
void calculate_phitilde_and_A_gauge_rhs(
      const double Lorenz_damping_factor,
      A_gauge_rhs_vars *restrict vars)
```
only takes two arguments. The first provides the damping factor for the generalized Lorenz gauge right-hand side contribution. The second serves as both the input and output. We already showed the struct in [Step 2](#interpolate_for_A_gauge_rhs), but we will give it again
```
typedef struct A_gauge_rhs_vars {
  double dxi[3];
  double alpha_interp;
  double alpha_Phi_minus_betaj_A_j_interp[4];
  double alpha_sqrtg_Ax_interp[2];
  double alpha_sqrtg_Ay_interp[2];
  double alpha_sqrtg_Az_interp[2];
  double phitildex[5], shiftx_interp[5];
  double phitildey[5], shifty_interp[5];
  double phitildez[5], shiftz_interp[5];
  double phitilde_rhs, A_x_gauge_rhs, A_y_gauge_rhs, A_z_gauge_rhs;
} A_gauge_rhs_vars;
```
This time, we will use all the elements of the struct. The inputs are all the elements except for the `*_rhs` variables. The element `dxi` is an array of the inverse grid spacing
$$
{\rm dxi[3]} = \{\frac{1}{dx}, \frac{1}{dy}, \frac{1}{dz}\}
$$
For a gridpoint $(i,j,k)$, the calculation actually takes place at $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$. For simplicity, we will refer to everything without the staggering offsets. `alpha_interp` is the $\alpha$ interpolated by `interpolate_for_A_gauge_rhs`. `alpha_Phi_minus_betaj_A_j_interp` is an array stencil with the points

|  Array Index  | $x$ Index  | $y$ Index  | $z$ Index  |
|:-------------:|:----------:|:----------:|:----------:|
|       0       |      i     |      j     |      k     |
|       1       |     i-1    |      j     |      k     |
|       2       |      i     |     j-1    |      k     |
|       3       |      i     |      j     |     k-1    |

All of the `alpha_sqrtg_A*_interp` arrays expect the points $i$ and $i+1$ in the direction of $A^i$ (i.e. for the point $(i,j,k)$ `alpha_sqrtg_Ax_interp` has the points $(i,j,k)$ and $(i+1,j,k)$). The arrays `phitilde*` and `shift*_interp` are also 1D stencils in the same direction as the variable with points from $i-2$ to $i+2$. As an example, `phitildey` and `shifty_interp` always have index $i$ and $k$, and the stencil goes from $j-2$ to $+2$.

Inside the actual `calculate_phitilde_and_A_gauge_rhs` function, we calculate the gauge terms of the $A_i$ right-hand sides using
\begin{align}
A_i^{\rm gauge rhs} &= -\partial_i \left( \alpha \Phi - \beta^j A_j \right) \\
&= -\frac{1}{\Delta x^i} \left( \Delta(\alpha \Phi - \beta^j A_j) \right)
\end{align}
which in terms of the struct variables gives
```
  vars->A_x_gauge_rhs = vars->dxi[0]*(vars->alpha_Phi_minus_betaj_A_j_interp[1]
                        - vars->alpha_Phi_minus_betaj_A_j_interp[0]);
  vars->A_y_gauge_rhs = vars->dxi[1]*(vars->alpha_Phi_minus_betaj_A_j_interp[2]
                        - vars->alpha_Phi_minus_betaj_A_j_interp[0]);
  vars->A_z_gauge_rhs = vars->dxi[2]*(vars->alpha_Phi_minus_betaj_A_j_interp[3]
                        - vars->alpha_Phi_minus_betaj_A_j_interp[0]);

```

Now, we just need to compute
$$
\tilde{\Phi}_{\rm RHS} = \mathrm{[shift\ advection\ term]} + \partial_j (\alpha \sqrt{\gamma} A^j)
$$
We first compute the shift advection term
$$
\tilde{\Phi}_{\rm advection\ RHS} = \partial_j (\beta^j \tilde{\Phi})
$$

The advection term depends on the direction of the shift at $(i,j,k)$. To simplify the description, we consider the $\beta^x$ contribution. If the shift is negative, then we use the stencil $i-2$ to $i$, otherwise we use $i$ to $i+2$. For negative $\beta^x$,
$$
\tilde{\Phi}_{{\rm RHS},\beta^x} = \frac{1}{\Delta x}\left( \beta^x_{i-2}\tilde{\Phi}_{i-2} -4\beta^x_{i-1}\tilde{\Phi}_{i-1} + 3\beta^x_{i}\tilde{\Phi}_{i}\right)
$$

For positive $\beta^x$,
$$
\tilde{\Phi}_{{\rm RHS},\beta^x} = \frac{1}{\Delta x}\left( -\beta^x_{i+2}\tilde{\Phi}_{i+2} +4\beta^x_{i+1}\tilde{\Phi}_{i+1} - 3\beta^x_{i}\tilde{\Phi}_{i}\right)
$$

The contributions from $\beta^y$ and $\beta^z$ are identical, but with the stencils in $j$ and $k$, respectively. After these, we add the
$$
\partial_j (\alpha \sqrt{\gamma} A^j)
$$
term, which we get directly from the `alpha_sqrtg_A*_interp` stencil. Finally, we compute the damping factor by adding
$$
- ({\rm Lorenz\_damping\_factor}) \alpha \tilde{\Phi}
$$

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log